<h1> Introduction

Installing and loading dependencies

In [71]:
!pip install -r requirement.txt

Keyring is skipped due to an exception: 'keyring.backends'


In [72]:
import awswrangler as wr
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


Read file from S3

In [73]:
prefix = 's3://'
bucket = 'glair-exploration-sagemaker-bucket'
files = ['ingredient.parquet.gz','menu_ingredient.parquet.gz','menu.parquet.gz',
         'order-*.parquet.gz','order_detail-*.parquet.gz','shop.parquet.gz']

df_ing = wr.s3.read_parquet(path=f'{prefix}{bucket}/{files[0]}')[['id','name']]
df_men_ing = wr.s3.read_parquet(path=f'{prefix}{bucket}/{files[1]}')
# df_men = wr.s3.read_parquet(path=f'{prefix}{bucket}/{files[2]}')
df_ord = wr.s3.read_parquet(path=f'{prefix}{bucket}/{files[3]}')[['id','shop_id','is_completed','created_time']]
df_ord_det = wr.s3.read_parquet(path=f'{prefix}{bucket}/{files[4]}')[['order_id','menu_id','quantity']]
df_shop = wr.s3.read_parquet(path=f'{prefix}{bucket}/{files[5]}')

df_shop.head()

,id,name,address,district,city,province,country,type,location_type,is_active,created_time,tags
0,0,shop name 16,shop address 16,Tambora,Jakarta Barat,DKI Jakarta,Indonesia,1,Office,1,2018-01-26,tags 7
1,1,shop name 5,shop address 5,Gambir,Jakarta Pusat,DKI Jakarta,Indonesia,1,Apartment,1,2018-02-07,"tags 0, tags 2"
2,2,shop name 19,shop address 19,Senen,Jakarta Pusat,DKI Jakarta,Indonesia,1,Apartment,1,2018-07-24,tags 9
3,3,shop name 4,shop address 4,Setiabudi,Jakarta Selatan,DKI Jakarta,Indonesia,1,Apartment,1,2018-08-06,tags 5
4,4,shop name 0,shop address 0,Koja,Jakarta Utara,DKI Jakarta,Indonesia,1,Public Space,1,2018-09-19,"tags 2, tags 0"


Limit only to shop_id that has highest order numbers

In [74]:
df_ord[['shop_id','id']].groupby('shop_id').agg('count').sort_values('id',ascending=False).head()

,id
shop_id,
1,123322
3,97004
6,81618
2,63996
7,53045


In [75]:
df_ord = df_ord[df_ord['shop_id']==1]
df_ord.sample(5)

,id,shop_id,is_completed,created_time
123575,123575,1,1,2020-02-06 16:00:53
48101,48101,1,1,2018-04-29 23:48:13
160007,160007,1,1,2020-12-13 23:17:28
144526,144526,1,1,2020-08-02 23:06:33
53037,53037,1,1,2018-06-10 23:22:01


In [76]:
len(df_ord)

123322

Analyze data range

In [77]:
df_ord.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123322 entries, 38568 to 161889
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            123322 non-null  Int64         
 1   shop_id       123322 non-null  Int64         
 2   is_completed  123322 non-null  Int64         
 3   created_time  123322 non-null  datetime64[ns]
dtypes: Int64(3), datetime64[ns](1)
memory usage: 5.1 MB


In [78]:
df_ord['created_time'].min()

Timestamp('2018-02-07 09:12:14')

In [79]:
df_ord['created_time'].max()

Timestamp('2020-12-30 20:58:57')

<h2> Prepare the dataset

In [80]:
df_ord.groupby('is_completed').agg('count')

,id,shop_id,created_time
is_completed,,,
0,252,252,252
1,123070,123070,123070


In [81]:
df_ord = df_ord[df_ord['is_completed']==1]
len(df_ord)

123070

In [82]:
df = df_ord.join(df_ord_det.set_index('order_id'), on='id')
df.head(10)

,id,shop_id,is_completed,created_time,menu_id,quantity
38568,38568,1,1,2018-02-07 12:41:00,15,1
38568,38568,1,1,2018-02-07 12:41:00,24,2
38568,38568,1,1,2018-02-07 12:41:00,30,1
38568,38568,1,1,2018-02-07 12:41:00,7,3
38569,38569,1,1,2018-02-07 20:17:58,12,1
38569,38569,1,1,2018-02-07 20:17:58,33,1
38569,38569,1,1,2018-02-07 20:17:58,26,1
38569,38569,1,1,2018-02-07 20:17:58,17,1
38569,38569,1,1,2018-02-07 20:17:58,27,1
38569,38569,1,1,2018-02-07 20:17:58,15,1


In [83]:
df = df.join(df_men_ing.set_index('menu_id'), on='menu_id')
df.head(10)

,id,shop_id,is_completed,created_time,menu_id,quantity,ingredient_id,amount
38568,38568,1,1,2018-02-07 12:41:00,15,1,24,40
38568,38568,1,1,2018-02-07 12:41:00,15,1,19,10
38568,38568,1,1,2018-02-07 12:41:00,15,1,33,150
38568,38568,1,1,2018-02-07 12:41:00,24,2,3,250
38568,38568,1,1,2018-02-07 12:41:00,24,2,5,2
38568,38568,1,1,2018-02-07 12:41:00,24,2,6,50
38568,38568,1,1,2018-02-07 12:41:00,30,1,4,250
38568,38568,1,1,2018-02-07 12:41:00,30,1,5,2
38568,38568,1,1,2018-02-07 12:41:00,30,1,6,50
38568,38568,1,1,2018-02-07 12:41:00,30,1,33,300


In [84]:
df = df.join(df_ing.set_index('id'), on='ingredient_id')
df.head(10)

,id,shop_id,is_completed,created_time,menu_id,quantity,ingredient_id,amount,name
38568,38568,1,1,2018-02-07 12:41:00,15,1,24,40,Grean Tea Bubuk
38568,38568,1,1,2018-02-07 12:41:00,15,1,19,10,Gula Pasir
38568,38568,1,1,2018-02-07 12:41:00,15,1,33,150,Air Mineral
38568,38568,1,1,2018-02-07 12:41:00,24,2,3,250,Nasi
38568,38568,1,1,2018-02-07 12:41:00,24,2,5,2,Telur
38568,38568,1,1,2018-02-07 12:41:00,24,2,6,50,Ayam
38568,38568,1,1,2018-02-07 12:41:00,30,1,4,250,Mie
38568,38568,1,1,2018-02-07 12:41:00,30,1,5,2,Telur
38568,38568,1,1,2018-02-07 12:41:00,30,1,6,50,Ayam
38568,38568,1,1,2018-02-07 12:41:00,30,1,33,300,Air Mineral


Drop unecessary column and null

In [85]:
df = df.drop(['id','shop_id','is_completed','menu_id','ingredient_id'], axis=1)
df.head()

,created_time,quantity,amount,name
38568,2018-02-07 12:41:00,1,40,Grean Tea Bubuk
38568,2018-02-07 12:41:00,1,10,Gula Pasir
38568,2018-02-07 12:41:00,1,150,Air Mineral
38568,2018-02-07 12:41:00,2,250,Nasi
38568,2018-02-07 12:41:00,2,2,Telur


In [86]:
df.isna().sum()

created_time        0
quantity            0
amount          42443
name            42443
dtype: int64

In [87]:
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df.isna().sum()

created_time    0
quantity        0
amount          0
name            0
dtype: int64

In [88]:
df = df.drop(['amount'], axis=1)
df.head()

,created_time,quantity,name
0,2018-02-07 12:41:00,1,Grean Tea Bubuk
1,2018-02-07 12:41:00,1,Gula Pasir
2,2018-02-07 12:41:00,1,Air Mineral
3,2018-02-07 12:41:00,2,Nasi
4,2018-02-07 12:41:00,2,Telur


In [89]:
df = df.groupby([df['created_time'].dt.strftime('%Y-%m-%d'),'name']).sum().reset_index()
df.groupby(['name']).sum().reset_index().sort_values('quantity', ascending=False).head(10)

,name,quantity
0,Air Mineral,343680
7,Gula Pasir,261317
30,Telur,123391
11,Kopi,110006
27,Susu Sapi,96437
29,Teh Hitam,82931
19,Mie,82506
3,Ayam,82369
10,Keju,69237
18,Meses Cokelat,55668


In [90]:
selected_ingredients = ['Air Mineral','Gula Pasir','Telur','Kopi','Susu Sapi']
df = df[df['name'].isin(selected_ingredients)]

In [91]:
df['days_of_week'] = df['created_time'].apply(lambda x: pd.Period(x,'D').day_of_week)
df['day'] = df['created_time'].apply(lambda x: pd.Period(x,'D').day)
df['month'] = df['created_time'].apply(lambda x: pd.Period(x,'D').month)
df['year'] = df['created_time'].apply(lambda x: pd.Period(x,'D').year)

In [92]:
final_df = df.drop('created_time', axis=1)
final_df.head(10)

,name,quantity,days_of_week,day,month,year
0,Air Mineral,162,2,7,2,2018
7,Gula Pasir,130,2,7,2,2018
11,Kopi,59,2,7,2,2018
27,Susu Sapi,41,2,7,2,2018
30,Telur,72,2,7,2,2018
32,Air Mineral,311,3,8,2,2018
39,Gula Pasir,214,3,8,2,2018
43,Kopi,73,3,8,2,2018
59,Susu Sapi,86,3,8,2,2018
62,Telur,149,3,8,2,2018


In [93]:
df.describe()

,quantity,days_of_week,day,month,year
count,5290.000000,5290.000000,5290.000000,5290.000000,5290.000000
mean,176.716635,2.999055,15.779773,6.704159,2019.034972
std,131.150077,1.999480,8.765106,3.354687,0.808647
min,14.000000,0.000000,1.000000,1.000000,2018.000000
25%,78.000000,1.000000,8.000000,4.000000,2018.000000
50%,140.000000,3.000000,16.000000,7.000000,2019.000000
75%,209.000000,5.000000,23.000000,10.000000,2020.000000
max,670.000000,6.000000,31.000000,12.000000,2020.000000


Define and execute Transformers

In [94]:
categorical_features = ["name"]

categorical_transformer = Pipeline(
    steps=[
        ("one", OneHotEncoder(drop='first', handle_unknown="error", sparse=False))
    ]
)
preprocess = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],
     remainder='passthrough'
)

In [95]:
final_df = df.drop('created_time', axis=1)
y = final_df.pop("quantity")
X_pre = preprocess.fit_transform(final_df)
y_pre = y.to_numpy().reshape(len(y), 1)

X = np.concatenate((y_pre, X_pre), axis=1)

Split data and save to S3

In [96]:
def filter_month(df, year, month):
    df = df[df['year']==year]
    
    return df[df['month']==month]
    
last_month = len(filter_month(final_df, 2020, 12))
last_two_month = len(filter_month(final_df, 2020, 11)) + last_month
print(last_month)
print(last_two_month)

150
300


In [97]:
train, validation, test = np.split(X, [len(X)-last_two_month, len(X)-last_month])

In [98]:
pd.DataFrame(train).to_csv(f"{prefix}{bucket}/train/train.csv", header=False, index=False)
pd.DataFrame(validation).to_csv(f"{prefix}{bucket}/validation/validation.csv", header=False, index=False)
pd.DataFrame(test).to_csv(f"{prefix}{bucket}/test/test.csv", header=False, index=False)

Test file

In [138]:
name_features = preprocess.transformers_[0][1]['one'].get_feature_names(categorical_features)

In [139]:
name_features=list(name_features)
name_features.insert(0,df.columns[2])
name_features=name_features+list(df.columns[3:])
name_features

['quantity',
 'name_Gula Pasir',
 'name_Kopi',
 'name_Susu Sapi',
 'name_Telur',
 'days_of_week',
 'day',
 'month',
 'year']

In [141]:
pd.DataFrame(test,columns=name_features).sample(10)

,quantity,name_Gula Pasir,name_Kopi,name_Susu Sapi,name_Telur,days_of_week,day,month,year
14,106,0.0,0.0,0.0,1.0,3.0,3.0,12.0,2020.0
142,41,0.0,1.0,0.0,0.0,1.0,29.0,12.0,2020.0
125,496,0.0,0.0,0.0,0.0,5.0,26.0,12.0,2020.0
27,148,0.0,1.0,0.0,0.0,6.0,6.0,12.0,2020.0
96,398,1.0,0.0,0.0,0.0,6.0,20.0,12.0,2020.0
40,144,0.0,0.0,0.0,0.0,2.0,9.0,12.0,2020.0
9,38,0.0,0.0,0.0,1.0,2.0,2.0,12.0,2020.0
60,515,0.0,0.0,0.0,0.0,6.0,13.0,12.0,2020.0
52,99,0.0,1.0,0.0,0.0,4.0,11.0,12.0,2020.0
22,162,0.0,1.0,0.0,0.0,5.0,5.0,12.0,2020.0
